# CLI Demo Interface

Quick command-line testing of the full RAG pipeline.

What we're doing:**

- Loading FAISS index and generator components
- Simple input loop for manual query testing
- Displaying answers with source references

What's Happening:

The input("\n🔍 Your question: ") function is pausing execution and waiting for you to type something and press Enter.

How to Use It:

- Look for a blinking cursor or input field below the output
- Type one of the example questions like: Which startups work on automotive and autonomous driving?
- Press Enter to run the query
- It will process and show you the results
- Then it will ask for another question until you type quit

In [ ]:
import sys
import os
import importlib.util

def import_all_components():
    current_dir = os.getcwd()
    
    # Retriever 
    retriever_path = os.path.join(current_dir, 'rag_components', 'faiss_retriever.py')
    spec = importlib.util.spec_from_file_location("faiss_retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Query Interface
    query_path = os.path.join(current_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_path)
    query_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_module)
    
    # Answer Generator
    answer_path = os.path.join(current_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_path)
    answer_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_module)
    
    return (retriever_module.FAISSRetriever, 
            query_module.SimpleQueryInterface,
            answer_module.RAGAnswerGenerator)

# Import components
FAISSRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_all_components()
print("✅ All components imported!")

VECTOR_INDEX_PATH = "../../04_models/vector_index"
retriever = FAISSRetriever(VECTOR_INDEX_PATH)
query_interface = SimpleQueryInterface(retriever)
answer_generator = RAGAnswerGenerator(query_interface)

In [ ]:
def working_cli_demo():
    print("\n" + "="*60)
    print("🚗 AUTOMOTIVE TECH RAG - WORKING CLI DEMO")
    print("="*60)
    print("Quick wins to demonstrate:")
    print("1. 'Which startups work on automotive and autonomous driving?'")
    print("2. 'What are the challenges and pain points in autonomous driving vehicles?'") 
    print("3. 'How is AI transforming the automotive industry?'")
    print("Type 'quit' to exit\n")
    
    while True:
        question = input("\n🔍 Your question: ").strip()
        if question.lower() in ['quit', 'exit', 'q']:
            break
            
        print("🔄 Searching through 19,038 document chunks...")
        result = answer_generator.generate_answer(question, top_k=4)
        
        print(f"\n🤖 ANSWER:")
        print(result['answer'])
        
        print(f"\n📚 SOURCES ({len(result['sources'])} found):")
        for i, source in enumerate(result['sources'][:3], 1):
            emoji = "🚀" if source['doc_type'] == 'startups_data' else "📄" if source['doc_type'] == 'research_paper' else "🔍"
            print(f"  {i}. {emoji} {source['source_file']}")
            print(f"     Score: {source['relevance_score']:.3f}")
            print(f"     Preview: {source['content_snippet'][:80]}...")

print("✅ CLI Demo ready - run 'working_cli_demo()' to test")

In [ ]:
working_cli_demo()

# Streamlit Generation and Optimization

Creating streamlit app and improving UX/UI (improvement of response speed and quality see notebook 03_generator_rag)

What we're doing:

- Create streamlit app
- Optimizing UX/UI

In [ ]:

def create_streamlit_app():
    streamlit_code = '''import streamlit as st
import sys
import os
sys.path.append('../03_notebooks')

# Use the same nuclear import for Streamlit
import importlib.util

def import_rag_components():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    parent_dir = os.path.join(current_dir, '..', '03_notebooks')
    
    # Import retriever
    retriever_path = os.path.join(parent_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Import query_interface  
    query_interface_path = os.path.join(parent_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_interface_path)
    query_interface_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_interface_module)
    
    # Import answer_generator
    answer_generator_path = os.path.join(parent_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_generator_path)
    answer_generator_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_generator_module)
    
    return (retriever_module.DocumentAwareRetriever, 
            query_interface_module.SimpleQueryInterface,
            answer_generator_module.RAGAnswerGenerator)

@st.cache_resource
def load_pipeline():
    DocumentAwareRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_rag_components()
    retriever = DocumentAwareRetriever("../04_models/vector_index")
    query_interface = SimpleQueryInterface(retriever)
    return RAGAnswerGenerator(query_interface)

st.set_page_config(page_title="Automotive Tech RAG", page_icon="🚗")
st.title("🚗 Automotive Tech RAG System")
st.markdown("Ask questions about automotive technology, startups, and research")

question = st.text_input("Enter your question:")

if st.button("Search") and question:
    answer_generator = load_pipeline()
    result = answer_generator.generate_answer(question, top_k=5)
    
    st.subheader("Answer")
    st.write(result["answer"])
    
    st.subheader(f"Sources ({len(result['sources'])} found)")
    for source in result["sources"]:
        with st.expander(f"{source['source_file']} (Score: {source['relevance_score']:.3f})"):
            st.write(f"**Type:** {source['doc_type']}")
            st.write(f"**Content:** {source['content_snippet']}")
'''
    
    with open("../05_app/streamlit_app.py", "w") as f:
        f.write(streamlit_code)
    print("✅ Streamlit app created: ../05_app/streamlit_app.py")

create_streamlit_app()